In [4]:
import numpy as np
from torch.utils.data import DataLoader
from model_slim import MahJongNetBatchedRevised
import torch.nn.functional as F
import torch
import os
import copy
import itertools
import matplotlib.pyplot as plt
import json
from torch import nn


# from model import DataEntryModule
from collections import defaultdict


def default_zero():
    return 0


plt.rcParams["font.sans-serif"] = ["SimHei"]
plt.rcParams["figure.figsize"] = (15, 7)
plt.rcParams["font.size"] = 16


# fmt:off
tile_list_raw = ['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B9',  # 饼
                 'W1', 'W2', 'W3', 'W4', 'W5', 'W6', 'W7', 'W8', 'W9',  # 万
                 'T1', 'T2', 'T3', 'T4', 'T5', 'T6', 'T7', 'T8', 'T9',  # 条
                 'F1', 'F2', 'F3', 'F4', 'J1', 'J2', 'J3'  # 风、箭
                 ]
fan_list_raw = ["大四喜", "大三元", "绿一色", "九莲宝灯", "四杠", "连七对", "十三幺", "清幺九", "小四喜", "小三元", "字一色", "四暗刻", "一色双龙会",
                "一色四同顺", "一色四节高", "一色四步高", "三杠", "混幺九", "七对", "七星不靠", "全双刻", "清一色", "一色三同顺", "一色三节高", "全大",
                "全中", "全小", "清龙", "三色双龙会", "一色三步高", "全带五", "三同刻", "三暗刻", "全不靠", "组合龙", "大于五", "小于五", "三风刻", "花龙",
                "推不倒", "三色三同顺", "三色三节高", "无番和", "妙手回春", "海底捞月", "杠上开花", "抢杠和",
                "碰碰和", "混一色", "三色三步高", "五门齐", "全求人", "双暗杠", "双箭刻", "全带幺",
                "不求人", "双明杠", "和绝张", "箭刻", "圈风刻", "门风刻", "门前清", "平和", "四归一",
                "双同刻", "双暗刻", "暗杠", "断幺", "一般高", "喜相逢", "连六", "老少副", "幺九刻",
                "明杠", "缺一门", "无字", "边张", "嵌张", "单钓将", "自摸"]
# fmt:on

In [5]:
def reversed_tile_conversion(list_rep, key_order_list):
    ret_dict = defaultdict(default_zero)
    for i in range(len(key_order_list)):
        ret_dict[key_order_list[i]] += list_rep[i]
    return ret_dict

In [6]:
def count_param(model):
    param_count = 0
    for param in model.parameters():
        param_count += param.view(-1).size()[0]
    return param_count


device = "cpu"
net = MahJongNetBatchedRevised(device).to(device)
count_param(net)

126

In [52]:
# change params here
log_name = "07_16_02_53_57-model_slim_rev6_normed2"
# log_name = "07_21_18_38_42-model_slim_sbot1_restricted"

# typically leave as is
logdir = "../log/"
log_suffix = "checkpoint"
extract_dir = "extracted_model_slim_params"
weight_list = {
    "top1": "best_acc.pkl",
    "top2": "best_acc_top2.pkl",
    "top3": "best_acc_top3.pkl",
}

# load model parameter from log
net.load_state_dict(
    torch.load(
        os.path.join(logdir, log_name, log_suffix, weight_list["top2"]),
        map_location=torch.device(device),
    )
)

<All keys matched successfully>

In [51]:
fan_output_human = reversed_tile_conversion(
    net.fan_coeff.detach().numpy(), fan_list_raw
)

In [53]:
fan_output_general = reversed_tile_conversion(
    net.fan_coeff.detach().numpy(), fan_list_raw
)

In [28]:
with open("statistics_general.json") as f:
    distribution = json.load(f)
sum_fan = sum(distribution.values())
distribution_general = defaultdict(default_zero)
for k in distribution.keys():
    distribution_general[k] = distribution[k] / sum_fan * 100

In [29]:
with open("statistics_#sbot1.json") as f:
    distribution = json.load(f)
sum_fan = sum(distribution.values())
distribution_human = defaultdict(default_zero)
for k in distribution.keys():
    distribution_human[k] = distribution[k] / sum_fan * 100

In [32]:
# The following is used to generate comparison between p2 and p3
dist_diff = {}
for k in distribution.keys():
    dist_diff[k] = distribution_human[k] - distribution_general[k]

In [38]:
dist_diff_sorted = sorted(dist_diff.items(), key=lambda item: -item[1])

In [65]:
sorted_dist_dict = defaultdict(default_zero)
for ent in dist_diff_sorted:
    sorted_dist_dict[ent[0]] = ent[1]

In [54]:
fan_diff = {}
for k in fan_output_general.keys():
    fan_diff[k] = fan_output_human[k] - fan_output_general[k]

In [56]:
fan_diff_sorted = sorted(fan_diff.items(), key=lambda item: -item[1])

In [59]:
sorted_fan_dict = {}
for ent in fan_diff_sorted:
    sorted_fan_dict[ent[0]] = ent[1]

In [63]:
sorted_dist_dict

{'花牌': 26.441393144952098,
 '全求人': 8.24735361839788,
 '缺一门': 4.2490248508918524,
 '明杠': 2.327854306652484,
 '无字': 2.148391132646491,
 '幺九刻': 2.136596030305677,
 '碰碰和': 1.0154679381081289,
 '和绝张': 0.8936430504712087,
 '双同刻': 0.6948347509839247,
 '四归一': 0.657028666945046,
 '杠上开花': 0.4115165677318359,
 '双明杠': 0.3229309548762948,
 '明暗杠': 0.16764088816050082,
 '抢杠和': 0.1439266782671583,
 '海底捞月': 0.12169098153664981,
 '一色三节高': 0.11046940818210213,
 '双箭刻': 0.07681786977482241,
 '门风刻': 0.0724825108746805,
 '圈风刻': 0.06096939833813875,
 '三色三节高': 0.049416467435190445,
 '三杠': 0.0403745737200992,
 '暗杠': 0.0390012404108534,
 '妙手回春': 0.028872006508648074,
 '三风刻': 0.011295117232864308,
 '一色四节高': 0.006352679589963476,
 '三同刻': 0.004448721144865282,
 '小四喜': 0.0034586518175215297,
 '双暗杠': 0.0029649353194859734,
 '全中': 0.00014181509408805418,
 '一色三同顺': -0.0009874329960711126,
 '四暗刻': -0.0009874329960711126,
 '混幺九': -0.0033168367234334755,
 '一般高': -0.004700441960452384,
 '全大': -0.009314978578176817,
 '全小': 

In [61]:
sorted_fan_dict["缺一门"]

0.008627901086583734

In [73]:
ordered_list = []
for f in sorted_fan_dict.keys():
    if (
        sorted_fan_dict[f] * sorted_dist_dict[f] > 0
        and abs(sorted_dist_dict[f]) > 1
        and abs(sorted_fan_dict[f]) > 0.02
    ):
        print(f, sorted_fan_dict[f], sorted_dist_dict[f])
        ordered_list.append(f)

全求人 0.03837879002094269 8.24735361839788
平和 -0.022124250885099173 -8.610641864378152
五门齐 -0.05007448198739439 -2.6008945278618314
三色三步高 -0.051378440111875534 -5.080452611922233
混一色 -0.05782352760434151 -1.039199235333794
清龙 -0.05981228733435273 -1.297062155991711
三色三同顺 -0.0705284234136343 -2.0953670118559256
花龙 -0.07963741989806294 -1.9424983470647195


In [87]:
fan_name = "碰碰和"
print(
    fan_output_general[fan_name],
    fan_output_human[fan_name],
    distribution_general[fan_name],
    distribution_human[fan_name],
)

0.05169188231229782 -0.002661252859979868 1.1185202333026554 2.1339881714107842


In [72]:
reverse_ordered_list = []
for f in sorted_fan_dict.keys():
    if (
        sorted_fan_dict[f] * sorted_dist_dict[f] < 0
        and abs(sorted_dist_dict[f]) > 1
        and abs(sorted_fan_dict[f]) > 0.02
    ):
        print(f, sorted_fan_dict[f], sorted_dist_dict[f])
        reverse_ordered_list.append(f)

自摸 0.021901805535890162 -3.0604452271365252
碰碰和 -0.05435313517227769 1.0154679381081289
